In [1]:
import pandas as pd

- UUID
- 產品
- 國家
- 月份
- 年
- 種類
- 數量
- 價格
- 退貨與否

分成

1. 產品表
- UUID
- 產品
- 種類

以及

2. 訂單表
- UUID_產品
- 國家
- 月份
- 年
- 數量
- 價格
- 退貨與否

In [2]:
df = pd.read_excel('data.xlsx')
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Month,Date,Season,TotalSales,IsReturned,category
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,12,2009-12-01,Winter,83.4,0,Decorations
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,12,2009-12-01,Winter,81.0,0,Lighting
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,12,2009-12-01,Winter,81.0,0,Lighting
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,12,2009-12-01,Winter,100.8,0,others
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,12,2009-12-01,Winter,30.0,0,Home Goods


In [3]:
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country', 'Month', 'Date', 'Season',
       'TotalSales', 'IsReturned', 'category'],
      dtype='object')

In [4]:
import pandas as pd
import uuid

# Assume your original dataframe 'df' has these columns:
# Description -> ProductName
# category -> Category
# InvoiceDate -> InvoiceDate
# Country -> Country
# Month -> Month
# Quantity -> Quantity
# Price -> Price
# IsReturned -> IsReturned

# Create product table (unique products and categories)
product_df = df[['Description', 'category']].drop_duplicates().copy()
product_df.rename(columns={'Description': 'ProductName', 'category': 'Category'}, inplace=True)
product_df['ProductUUID'] = [str(uuid.uuid4()) for _ in range(len(product_df))]

# Save product table
product_df[['ProductUUID', 'ProductName', 'Category']].to_excel("products.xlsx", index=False)

# Create mapping: ProductName -> ProductUUID
product_uuid_map = dict(zip(product_df['ProductName'], product_df['ProductUUID']))

# Create sales table
sales_df = df.copy()
sales_df.rename(columns={'Description': 'ProductName', 'category': 'Category', 'InvoiceDate': 'InvoiceDate'}, inplace=True)
sales_df['ProductUUID'] = sales_df['ProductName'].map(product_uuid_map)
sales_df['Year'] = pd.to_datetime(sales_df['InvoiceDate']).dt.year

# Select columns for output
sales_output = sales_df[['ProductUUID', 'Country', 'Month', 'Year', 'Quantity', 'Price', 'IsReturned']]

# Save sales table
sales_output.to_excel("sales.xlsx", index=False)


# 分割後

In [5]:
product_test = pd.read_excel('products.xlsx')
sales_test = pd.read_excel('sales.xlsx')

In [6]:
product_test.head()

,ProductUUID,ProductName,Category
0,f4a4314f-c4ac-4ad2-8374-472f521b12fd,15CM CHRISTMAS GLASS BALL 20 LIGHTS,Decorations
1,d4123154-9968-4c9c-8dd3-d2ace9ac5ab0,PINK CHERRY LIGHTS,Lighting
2,0d576d9b-4f8c-4235-a96f-9a0a3f7328d3,WHITE CHERRY LIGHTS,Lighting
3,30e40214-eb23-41a4-8184-68437a1323f1,"RECORD FRAME 7"" SINGLE SIZE",others
4,dec6bd8f-6e1f-42e7-90e3-7e15ee99cb56,STRAWBERRY CERAMIC TRINKET BOX,Home Goods


In [7]:
sales_test.head()

,ProductUUID,Country,Month,Year,Quantity,Price,IsReturned
0,f4a4314f-c4ac-4ad2-8374-472f521b12fd,United Kingdom,12,2009,12,6.95,0
1,d4123154-9968-4c9c-8dd3-d2ace9ac5ab0,United Kingdom,12,2009,12,6.75,0
2,0d576d9b-4f8c-4235-a96f-9a0a3f7328d3,United Kingdom,12,2009,12,6.75,0
3,30e40214-eb23-41a4-8184-68437a1323f1,United Kingdom,12,2009,48,2.10,0
4,dec6bd8f-6e1f-42e7-90e3-7e15ee99cb56,United Kingdom,12,2009,24,1.25,0


# CustomerID to Country

In [3]:
customer_country_map = (
    df.groupby(['Customer ID', 'Country'])
    .size()
    .reset_index(name='count')
    .sort_values(['Customer ID', 'count'], ascending=[True, False])
    .drop_duplicates(subset='Customer ID')
    .drop(columns='count')
)

# Rename columns for clarity
customer_country_map.columns = ['Customer ID', 'Country']

# Save to CSV
customer_country_map.to_excel("customer_country_mapping.xlsx", index=False)

print("✅ Saved customer-to-country mapping to 'customer_country_mapping.xlsx'")

customer_country_map.head()


✅ Saved customer-to-country mapping to 'customer_country_mapping.xlsx'


,Customer ID,Country
0,12346,United Kingdom
1,12347,Iceland
2,12348,Finland
3,12349,Italy
4,12351,Unspecified


# Product to Price

In [ ]:
import pandas as pd

# Load the Excel file
df = pd.read_excel('data.xlsx')

# Drop duplicates to get unique product-price mapping
product_price_df = df[['Description', 'Price']].drop_duplicates()

# Export to CSV
product_price_df.to_csv('product_price_map.csv', index=False)

print("✅ Exported product_price_map.csv")


✅ Exported product_price_map.csv


In [8]:
product_price_df.head()

,Description,Price
0,15CM CHRISTMAS GLASS BALL 20 LIGHTS,6.95
1,PINK CHERRY LIGHTS,6.75
2,WHITE CHERRY LIGHTS,6.75
3,"RECORD FRAME 7"" SINGLE SIZE",2.10
4,STRAWBERRY CERAMIC TRINKET BOX,1.25
